In [2]:
import ipywidgets as widgets
model_no = widgets.Text(value='4.0')
epochs = widgets.IntSlider(value=300, min=0, max=10000, step=100)
print('Model number')
display(model_no)
print('Epochs')
display(epochs)

Model number


Text(value='4.0')

Epochs


IntSlider(value=300, max=10000, step=100)

##### CNN + GRU, 
##### Conservative Data augmentation methods - zoom , horizontal flip, rotation, (change zoom to 0.4., width shift range = 0.4, height_shift = 0.4 rotation range 12)

sequence generator shape (corrected) (b, n, w, h, c)

In [3]:
import os
import glob
import keras

In [4]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf
import datetime

In [5]:
# from patrice's blogpost
from keras_video import VideoFrameGenerator

In [6]:
classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()
print(classes)

['hand_ties', 'suture_throws', 'thread_cuts']


In [7]:
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

In [8]:
# pattern to get videos and classes
glob_pattern='../../data_v3_model_2.2/model_data/{classname}/*.avi'

In [9]:
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.4,
    horizontal_flip=True,
    rotation_range=12,
    width_shift_range=.4,
    height_shift_range=.4)

In [10]:
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.33, 
    shuffle=False,
    transformation=data_aug,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 121, train count: 247
class suture_throws, validation count: 122, train count: 248
class thread_cuts, validation count: 121, train count: 248
Total data: 3 classes for 743 files for train


In [11]:
# getting validation data
valid = train.get_validation_generator()

Total data: 3 classes for 364 files for validation


In [12]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

In [13]:
#valid.files

## BUILD CONV NET

In [14]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_mobilenet(shape=(224, 224, 3), nbout=3):
#     model = keras.applications.mobilenet.MobileNet(
#         include_top=False,
#         input_shape=shape,
#         weights='imagenet')
    
    model = tf.keras.applications.ResNet50(
        include_top=False,
        weights="imagenet",
        #input_tensor=None,
        input_shape=shape,
        pooling=None,
        classes=1000,
    )
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
        
    # adding a max pool
    output = GlobalMaxPool2D()

    return keras.Sequential([model, output])

## Build GRU

In [15]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout
# Shape (5, 112, 112, 3) 5 - time sequence length 112x112 = height vs width 3 - num channels
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    # KEY = allows you to add a time sequence to a layer one at a time
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

## this is where you tell the model how to train - loss function, weight update mechanism

In [16]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (20, 112, 112, 3)
# action model - GRU set up for Time shifted CNN
model = action_model(INSHAPE, len(classes))

# this is where you tell the model how to train - loss function, weight update mechanism
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

## epochs, call backs

In [17]:
log_dir = "logs/fit/model_{}_".format(model_no.value) + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
print('logs for this run are here: {}'.format(log_dir))

logs for this run are here: logs/fit/model_4.0_20210617-150118


In [18]:
EPOCHS=epochs.value
print('number of epochs: {}'.format(epochs.value))
model_dir = './model_' + model_no.value + '_chkp/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    print('new folder created for new model: {}'.format(model_dir))
    
model_save_path = model_dir + 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'    

number of epochs: 300


In [19]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        model_save_path,
        verbose=1),
    tensorboard_callback
]

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

Epoch 1/300


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/time_distributed/resnet50/conv1_conv/Conv2D (defined at <ipython-input-19-b7e4ac8294b2>:10) ]] [Op:__inference_train_function_16394]

Function call stack:
train_function
